In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,2021-02-06 17:47:44,62.4560,-114.3525,-43.01,66,1,3.44,mist
1,1,Longyearbyen,SJ,2021-02-06 17:51:47,78.2186,15.6401,23.00,74,75,3.44,broken clouds
2,2,Upernavik,GL,2021-02-06 17:51:47,72.7868,-56.1549,19.96,80,100,4.99,overcast clouds
3,3,Havelock,US,2021-02-06 17:51:17,34.8791,-76.9013,50.00,57,75,8.05,broken clouds
4,4,Manthani,IN,2021-02-06 17:51:47,18.6500,79.6667,70.09,34,0,2.10,clear sky


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Date                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 150


In [5]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Manthani,IN,2021-02-06 17:51:47,18.6500,79.6667,70.09,34,0,2.10,clear sky
7,7,Castro,BR,2021-02-06 17:51:48,-24.7911,-50.0119,76.44,48,0,6.96,clear sky
14,14,East London,ZA,2021-02-06 17:49:22,-33.0153,27.9116,75.20,94,20,4.61,few clouds
15,15,Alofi,NU,2021-02-06 17:51:49,-19.0595,-169.9187,77.00,88,100,4.61,overcast clouds
18,18,Rikitea,PF,2021-02-06 17:51:49,-23.1203,-134.9692,77.65,74,100,12.37,overcast clouds
19,19,Georgetown,MY,2021-02-06 17:50:02,5.4112,100.3354,82.40,83,20,2.30,few clouds
21,21,Warmbad,NaN,2021-02-06 17:51:49,-28.4500,18.7333,88.21,21,0,20.69,clear sky
24,24,Tucuma,BR,2021-02-06 17:51:50,-0.1833,-50.0500,82.90,80,92,7.16,overcast clouds
30,30,Dhidhdhoo,MV,2021-02-06 17:51:51,6.8833,73.1000,81.57,77,53,11.45,broken clouds
31,31,Vaitape,PF,2021-02-06 17:51:51,-16.5167,-151.7500,81.45,70,0,10.40,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                  0
City                     0
Country                 30
Date                   794
Lat                      0
Lng                      0
Max Temp                 0
Humidity                 0
Cloudiness               0
Wind Speed               0
Current Description      0
dtype: int64

In [7]:
preferred_cities_df.notnull().sum()

City_ID                1324
City                   1324
Country                1294
Date                    530
Lat                    1324
Lng                    1324
Max Temp               1324
Humidity               1324
Cloudiness             1324
Wind Speed             1324
Current Description    1324
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Manthani,IN,70.09,clear sky,18.6500,79.6667,
7,Castro,BR,76.44,clear sky,-24.7911,-50.0119,
14,East London,ZA,75.20,few clouds,-33.0153,27.9116,
15,Alofi,NU,77.00,overcast clouds,-19.0595,-169.9187,
18,Rikitea,PF,77.65,overcast clouds,-23.1203,-134.9692,
19,Georgetown,MY,82.40,few clouds,5.4112,100.3354,
24,Tucuma,BR,82.90,overcast clouds,-0.1833,-50.0500,
30,Dhidhdhoo,MV,81.57,broken clouds,6.8833,73.1000,
31,Vaitape,PF,81.45,light rain,-16.5167,-151.7500,
36,Barra Dos Coqueiros,BR,86.00,clear sky,-10.9089,-37.0386,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Manthani,IN,70.09,clear sky,18.6500,79.6667,Social Welfare Boys Hostel
7,Castro,BR,76.44,clear sky,-24.7911,-50.0119,CHACARA BAILLY
14,East London,ZA,75.20,few clouds,-33.0153,27.9116,Tu Casa
15,Alofi,NU,77.00,overcast clouds,-19.0595,-169.9187,Taloa Heights
18,Rikitea,PF,77.65,overcast clouds,-23.1203,-134.9692,Pension Maro'i
19,Georgetown,MY,82.40,few clouds,5.4112,100.3354,Cititel Penang
24,Tucuma,BR,82.90,overcast clouds,-0.1833,-50.0500,
30,Dhidhdhoo,MV,81.57,broken clouds,6.8833,73.1000,Dhidhoo beach
31,Vaitape,PF,81.45,light rain,-16.5167,-151.7500,Conrad Bora Bora Nui
36,Barra Dos Coqueiros,BR,86.00,clear sky,-10.9089,-37.0386,Quality Hotel Aracaju


In [12]:
# 7. Drop the rows where there is no Hotel Name.

new_hotel_df= hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = new_hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Manthani,IN,70.09,clear sky,18.6500,79.6667,Social Welfare Boys Hostel
7,Castro,BR,76.44,clear sky,-24.7911,-50.0119,CHACARA BAILLY
14,East London,ZA,75.20,few clouds,-33.0153,27.9116,Tu Casa
15,Alofi,NU,77.00,overcast clouds,-19.0595,-169.9187,Taloa Heights
18,Rikitea,PF,77.65,overcast clouds,-23.1203,-134.9692,Pension Maro'i
...,...,...,...,...,...,...,...
3414,Hobyo,SO,77.72,clear sky,5.3505,48.5268,Halane Hotel and Restaurent
3417,Kloulklubed,PW,78.80,broken clouds,7.0419,134.2556,Storyboard Beach Resort
3418,Tual,ID,83.86,light rain,-5.6667,132.7500,Grand Vilia Hotel
3419,Nacala,MZ,79.48,overcast clouds,-14.5428,40.6728,Nacala Plaza Business Design Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))